<a href="https://colab.research.google.com/github/Uthmanic/DSN-AI-Bootcamp-2020/blob/master/(Uthman_Tijani)_DSN_AI_Bootcamp_Qualification_Hackathon_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/Uthmanic/DSN-AI-Bootcamp-2020/master/Train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Uthmanic/DSN-AI-Bootcamp-2020/master/Test.csv')
train_copy = train.copy()
test_copy = test.copy()

In [3]:
#train_copy.isnull().sum()

In [4]:
#test_copy.isnull().sum()

In [5]:
test_copy.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,0.0,1649749.0,5446.0,0.0,48187375,114686147.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1086.0,6204.0,2676.0,60.0,14691.0,9794.0,NaN,238.0,1.8334,0.0,NaN,22.0,2.0,14.0,0.0,NaN,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,622789.0,1025793.0,1396510.0,34694910,87479487.0,1.0,1.0,1.0,2.0,2.0,142.560,NaN,NaN,570.585,3082.5,10221.0,6814.0,60.0,11955.0,3528.0,NaN,1216.0,19.5000,3.0,NaN,30.0,18.0,14.0,2.0,NaN,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,0.0,118256.0,NaN,NaN,21973443,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.0,4836.0,NaN,NaN,NaN,7.8334,0.0,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1391064.0,1176725.0,1220293.0,16576457,NaN,2.0,2.0,2.0,3.0,3.0,109.005,13050.00,NaN,109.005,1414.5,4197.0,2798.0,60.0,14964.0,NaN,NaN,248.0,9.0000,1.0,1.0,36.0,0.0,26.0,0.0,NaN,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,271133.0,613064.0,4779643.0,48187375,72049802.0,0.0,0.0,0.0,3.0,3.0,139.065,469.29,469.29,15.180,2724.0,9855.0,6570.0,182.0,11679.0,5900.0,NaN,2388.0,16.6666,1.0,0.0,24.0,28.0,20.0,0.0,NaN,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [6]:
train_copy.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [7]:
 train_copy = train_copy.fillna(-999)
test_copy = test_copy.fillna(-999)

In [8]:
#train_copy.isnull().sum()

In [9]:
#test_copy.isnull().sum()

In [10]:
from sklearn.preprocessing import LabelEncoder
 
train_copy["form_field47"] = LabelEncoder().fit_transform(train_copy["form_field47"])
train_copy["default_status"] = LabelEncoder().fit_transform(train_copy["default_status"])
test_copy["form_field47"] = LabelEncoder().fit_transform(test_copy["form_field47"])

In [11]:
train_copy.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,-999.00,-999.00,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,-999.0,1052.0,8.0000,2.0,-999.0,12.0,12.0,4.0,0.0,-999.0000,-999.0,0.392854,2.02,0.711632,0.0,0.0,0,-999.000000,1.129518,0.044335,0
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,-999.0,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,-999.0000,-999.0,0.314281,8.08,0.183584,-999.0,0.0,0,349.805730,1.620483,0.322436,0
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,-999.0,956940.0,-999.0,192944.0,1079864.0,-999.0,-999.0,0.0,16576457,22452691.0,-999.0,-999.0,-999.0,0.0,0.0,-999.000,-999.00,-999.00,-999.000,1597.5,4746.0,3164.0,60.0,15696.0,-999.0,14508.0,70.0,0.6666,0.0,-999.0,12.0,0.0,6.0,0.0,149.6805,-999.0,0.162965,18.18,0.791136,0.0,0.0,0,-999.000000,1.513370,0.011640,1
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,-999.0,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,-999.0,-999.0,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,-999.0000,-999.0,0.488884,2.02,0.685168,-999.0,0.0,0,89.940100,0.664452,0.082729,0
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,-999.0,0.275000,12.12,0.438168,0.0,0.0,0,97.887502,1.427891,0.045630,0


In [12]:
#model feature and target
X = train_copy.drop(["Applicant_ID", "default_status","form_field48",'form_field49'], axis =1)
y = train_copy["default_status"]
 
test_data = test_copy.drop(["Applicant_ID","form_field48",'form_field49'], axis = 1)

In [13]:
!pip install catboost

     |████████████████████████████████| 66.1MB 49kB/s 


In [14]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold

In [15]:
new = []
predi = []
 
fold = StratifiedKFold(n_splits=55, shuffle=True, random_state=101)
 
for train_index, test_index in fold.split(X,y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y[train_index],y[test_index]
 
  catb = CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                          bootstrap_type='Bayesian',use_best_model=True)
 
  catb.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_test,y_test)],
           early_stopping_rounds=100,verbose=100)
  
  catb_pred = catb.predict_proba(X_test)[:,1]
 
  new.append(roc_auc_score(y_test,catb_pred))
 
  catpred = catb.predict_proba(test_data)[:,1]
  predi.append(catpred)

0:	test: 0.7760208	test1: 0.7635501	best: 0.7635501 (0)	total: 89.1ms	remaining: 1m 28s
100:	test: 0.8404253	test1: 0.8290143	best: 0.8290143 (100)	total: 3.52s	remaining: 31.3s
200:	test: 0.8468223	test1: 0.8298153	best: 0.8301378 (159)	total: 6.97s	remaining: 27.7s
300:	test: 0.8517080	test1: 0.8300546	best: 0.8304707 (220)	total: 10.4s	remaining: 24.1s
400:	test: 0.8556863	test1: 0.8310169	best: 0.8319532 (364)	total: 13.8s	remaining: 20.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.831953186
bestIteration = 364

Shrink model to first 365 iterations.
0:	test: 0.7786346	test1: 0.7862211	best: 0.7862211 (0)	total: 34.7ms	remaining: 34.6s
100:	test: 0.8398043	test1: 0.8393186	best: 0.8393446 (97)	total: 3.44s	remaining: 30.6s
200:	test: 0.8470947	test1: 0.8432770	best: 0.8436047 (185)	total: 6.85s	remaining: 27.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8436046814
bestIteration = 185

Shrink model to first 186 iterations.
0:	test: 

In [16]:
np.mean(new)

0.8419633487698279

In [18]:
lgbm_new = []
lgbm_predi = []
 
fold = StratifiedKFold(n_splits=55, shuffle=True, random_state=101)
 
for train_index, test_index in fold.split(X,y):
  x_train, x_test = X.iloc[train_index], X.iloc[test_index]
  Y_train, Y_test = y[train_index],y[test_index]
 
  lgbm = LGBMClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                          bootstrap_type='Bayesian',use_best_model=True)
 
  lgbm.fit(x_train, Y_train, eval_set=[(x_train,Y_train),(x_test,Y_test)],
           early_stopping_rounds=100,verbose=100)
  
  lgbm_pred = lgbm.predict_proba(x_test)[:,1]
 
  lgbm_new.append(roc_auc_score(Y_test,lgbm_pred))
 
  lgbm_pred = lgbm.predict_proba(test_data)[:,1]
  lgbm_predi.append(lgbm_pred)

Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.391638	valid_1's binary_logloss: 0.418645
[200]	training's binary_logloss: 0.377203	valid_1's binary_logloss: 0.418149
[300]	training's binary_logloss: 0.3658	valid_1's binary_logloss: 0.418056
Early stopping, best iteration is:
[225]	training's binary_logloss: 0.374125	valid_1's binary_logloss: 0.417747
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.391302	valid_1's binary_logloss: 0.403676
[200]	training's binary_logloss: 0.376949	valid_1's binary_logloss: 0.40215
Early stopping, best iteration is:
[164]	training's binary_logloss: 0.381525	valid_1's binary_logloss: 0.401474
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.391204	valid_1's binary_logloss: 0.408874
[200]	training's binary_logloss: 0.376581	valid_1's binary_logloss: 0.409761
Early stopping, best iteration is:
[145]	training's

In [19]:
np.mean(lgbm_new)

0.8404349087166137

In [21]:
weighted_average = ((lgbm_pred * 0.4) + (catpred * 0.6))
weighted_average

array([0.34107648, 0.32114449, 0.41647638, ..., 0.22684173, 0.48819583,
       0.19724135])

In [22]:
pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": weighted_average}).to_csv("73_submission.csv", index = False)

In [23]:
from google.colab import files
files.download('73_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
sub73 = pd.read_csv('73_submission.csv')

In [26]:
weighted_average2 = (((catpred * 0.3 + sub73.default_status * 0.7) * 0.7) + (lgbm_pred * 0.3))

In [27]:
pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": weighted_average2}).to_csv("74_submission.csv", index = False)

In [28]:
from google.colab import files
files.download('74_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
sub74 = pd.read_csv('74_submission.csv')

In [30]:
weighted_average3 = (((sub74.default_status * 0.4 + sub73.default_status * 0.6) * 0.8) + (catpred * 0.2))

In [31]:
pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": weighted_average3}).to_csv("75_submission.csv", index = False)

In [32]:
from google.colab import files
files.download('75_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
sub75 = pd.read_csv('75_submission.csv')

weighted_average4 = (((sub75.default_status * 0.7 + sub73.default_status * 0.3) * 0.8) + (catpred * 0.2))

pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": weighted_average4}).to_csv("76_submission.csv", index = False)

from google.colab import files
files.download('76_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
sub76 = pd.read_csv('76_submission.csv')

weighted_average5 = (((sub75.default_status * 0.7 + sub76.default_status * 0.3) * 0.7) + (sub73.default_status * 0.3))

pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": weighted_average5}).to_csv("77_submission.csv", index = False)

from google.colab import files
files.download('77_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:

weighted_average6 = ((sub75.default_status * 0.5) + (lgbm_pred * 2) + (catpred * 0.3))

pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": weighted_average6}).to_csv("79_submission.csv", index = False)

from google.colab import files
files.download('79_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
best_params={'bagging_temperature': 0.41010395885331385,
 'border_count': 186,
 'depth': 8,
 'iterations': 323,
 'l2_leaf_reg': 21,
 'learning_rate': 0.0673344419215237,
 'random_strength': 3.230824361824754e-06,
 'scale_pos_weight': 0.7421091918485163}

In [16]:
best_params['iterations']=1000

In [25]:

new_tuned = []
predi_tuned = []
 
fold = StratifiedKFold(n_splits=1001, shuffle=True, random_state=101)
 
for train_index, test_index in fold.split(X,y):
  Xtrain, Xtest = X.iloc[train_index], X.iloc[test_index]
  ytrain, ytest = y[train_index],y[test_index]
 
  tuned_model = CatBoostClassifier(**best_params,task_type = "GPU",od_type='Iter',one_hot_max_size=10)

 
  tuned_model.fit(Xtrain, ytrain, eval_set=[(Xtest,ytest)],
           early_stopping_rounds=100,verbose=100)
  
  tuned_pred = tuned_model.predict_proba(Xtest)[:,1]
 
  new_tuned.append(roc_auc_score(ytest,tuned_pred))
 
  tuned_test = tuned_model.predict_proba(test_data)[:,1]
  predi_tuned.append(tuned_test)

Streaming output truncated to the last 5000 lines.
bestTest = 0.400752978
bestIteration = 46
Shrink model to first 47 iterations.
0:	learn: 0.6394747	test: 0.6459966	best: 0.6459966 (0)	total: 14.5ms	remaining: 14.5s
100:	learn: 0.3471196	test: 0.3764079	best: 0.3763712 (94)	total: 1.41s	remaining: 12.6s
bestTest = 0.3763712081
bestIteration = 94
Shrink model to first 95 iterations.
0:	learn: 0.6387981	test: 0.6307413	best: 0.6307413 (0)	total: 14.8ms	remaining: 14.8s
100:	learn: 0.3473518	test: 0.3388944	best: 0.3383249 (84)	total: 1.48s	remaining: 13.2s
200:	learn: 0.3319093	test: 0.3387357	best: 0.3369515 (114)	total: 2.87s	remaining: 11.4s
bestTest = 0.3369514667
bestIteration = 114
Shrink model to first 115 iterations.
0:	learn: 0.6389064	test: 0.6339180	best: 0.6339180 (0)	total: 14.9ms	remaining: 14.9s
100:	learn: 0.3475132	test: 0.3484790	best: 0.3481554 (97)	total: 1.42s	remaining: 12.6s
200:	learn: 0.3319198	test: 0.3418279	best: 0.3416468 (199)	total: 2.79s	remaining: 11.1s


In [26]:
np.mean(new_tuned)

0.8456510844612776

In [27]:

pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": tuned_test}).to_csv("81_submission.csv", index = False)

from google.colab import files
files.download('81_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
from google.colab import files
uploaded = files.upload()

Saving 75_submission.csv to 75_submission.csv
Saving 76_submission.csv to 76_submission.csv
Saving 77_submission.csv to 77_submission.csv
Saving 78_submission.csv to 78_submission.csv


In [31]:
import io
sub75 = pd.read_csv(io.BytesIO(uploaded['75_submission.csv']))
sub76 = pd.read_csv(io.BytesIO(uploaded['76_submission.csv']))
sub77 = pd.read_csv(io.BytesIO(uploaded['77_submission.csv']))
sub78 = pd.read_csv(io.BytesIO(uploaded['78_submission.csv']))

In [32]:
avg = (sub75.default_status + sub76.default_status + sub77.default_status + sub78.default_status) / 4

In [33]:

pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": avg}).to_csv("82_submission.csv", index = False)

from google.colab import files
files.download('82_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
sub82 = pd.read_csv('82_submission.csv')
sub83 = pd.read_csv('83_submission.csv')
sub85 = pd.read_csv('85_submission.csv')

In [42]:
weighted_average7 = ((sub85.default_status * 0.8) + (sub82.default_status* 1) + (sub83.default_status * 0.1))


pd.DataFrame({"Applicant_ID": test_copy["Applicant_ID"], "default_status": weighted_average7}).to_csv("87_submission.csv", index = False)

from google.colab import files
files.download('87_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>